In [2]:
"""
	Autores:
	- Astrid Alvarado
	- Junior Lara
"""

# Asegurese de seleccionar el Kernel de Python 3, puede depender de su entorno.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, count, when, first, explode, split

In [3]:
spark = SparkSession\
    .builder\
    .appName("Solver problema 3 - DF")\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/01 01:50:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# ITEM 1: Ver el comportamiento de la popularidad en función de la distribución.
# Archivo necesiario: final_animedataset.csv
# Campos necesarios:
# 	- title
# 	- gender
# 	- popularity

# Direccionio del archivo
# Ej. file = "../../tests/vshort-final_animedataset.csv"
file = "../../tests/vshort-final_animedataset.csv"

final_animedataset = spark\
	.read\
	.format("csv")\
	.option("header", "true")\
	.load(file)

final_animedataset.show(20)

+--------+--------+--------+-------+------+--------------------+----+------------+-----+---------+------+----------+--------------------+
|username|anime_id|my_score|user_id|gender|               title|type|      source|score|scored_by|  rank|popularity|               genre|
+--------+--------+--------+-------+------+--------------------+----+------------+-----+---------+------+----------+--------------------+
|karthiga|      21|       9|2255153|Female|           One Piece|  TV|       Manga| 8.54|   423868|  91.0|        35|Action, Adventure...|
|karthiga|      59|       7|2255153|Female|             Chobits|  TV|       Manga| 7.53|   175388|1546.0|       188|Sci-Fi, Comedy, D...|
|karthiga|      74|       7|2255153|Female|        Gakuen Alice|  TV|       Manga| 7.77|    33244| 941.0|      1291|Comedy, School, S...|
|karthiga|     120|       7|2255153|Female|       Fruits Basket|  TV|       Manga| 7.77|   167968| 939.0|       222|Slice of Life, Co...|
|karthiga|     178|       7|225515

In [5]:
# Calcular la cantidad de usuarios por género y la popularidad para cada título
count_female_male_by_title = final_animedataset.groupBy("title") \
           .agg(
               count(when(col("gender") == "Female", 1)).alias("female_count"),  # Contar las females
               count(when(col("gender") == "Male", 1)).alias("male_count"),      # Contar los males
               first("popularity").alias("popularity")                          # Tomar la popularidad única
           )

# Agregar las columnas de porcentaje para hombres y mujeres
count_female_male_by_title_with_percen = count_female_male_by_title.withColumn(
    "female_percentage", round((col("female_count") / (col("female_count") + col("male_count")) * 100), 2)
).withColumn(
    "male_percentage", round((col("male_count") / (col("female_count") + col("male_count")) * 100), 2)
)

# Mostrar los resultados
count_female_male_by_title_with_percen.show(20)

+--------------------+------------+----------+----------+-----------------+---------------+
|               title|female_count|male_count|popularity|female_percentage|male_percentage|
+--------------------+------------+----------+----------+-----------------+---------------+
|"""Bungaku Shoujo...|           1|         0|      1556|            100.0|            0.0|
|"Gyakuten Saiban:...|           1|         1|    5839.0|             50.0|           50.0|
|"Naruto Shippuude...|           1|         0|      1771|            100.0|            0.0|
|        .hack//Roots|           0|         1|      1391|              0.0|          100.0|
|         .hack//Sign|           0|         1|       650|              0.0|          100.0|
|.hack//Tasogare n...|           0|         1|      1474|              0.0|          100.0|
|              11eyes|           1|         0|       487|            100.0|            0.0|
|12-sai.: Chicchan...|           1|         0|      2789|            100.0|     

In [6]:
# ITEM 2: Ver la tendencia de géneros de anime en función de la distribución.
# Archivo necesiario: final_animedataset.csv
# Campos necesarios:
# 	- gender
# 	- genre

# Separar los géneros en la columna "genre" y explotar la lista en filas individuales
exploded_genres = final_animedataset.withColumn("genre", explode(split(col("genre"), ", ")))

# Contar la cantidad de usuarios por género (Female y Male) para cada género de anime
count_female_male_by_genres = exploded_genres.groupBy("genre") \
           .agg(
               count(when(col("gender") == "Female", 1)).alias("female_count"),  # Contar las females
               count(when(col("gender") == "Male", 1)).alias("male_count")       # Contar los males
           )

# Agregar las columnas de porcentaje para hombres y mujeres con redondeo a 2 decimales
count_female_male_by_genres_with_percen = count_female_male_by_genres.withColumn(
    "female_percentage", round((col("female_count") / (col("female_count") + col("male_count")) * 100), 2)
).withColumn(
    "male_percentage", round((col("male_count") / (col("female_count") + col("male_count")) * 100), 2)
)

# Mostrar los resultados
count_female_male_by_genres_with_percen.show(20)

+------------+------------+----------+-----------------+---------------+
|       genre|female_count|male_count|female_percentage|male_percentage|
+------------+------------+----------+-----------------+---------------+
|      Seinen|          82|        16|            83.67|          16.33|
|     Romance|         241|        58|             80.6|           19.4|
|    Dementia|          18|         7|             72.0|           28.0|
|       Magic|          60|        26|            69.77|          30.23|
|    Thriller|          40|        15|            72.73|          27.27|
|       Josei|          21|         4|             84.0|           16.0|
|   Adventure|         124|        78|            61.39|          38.61|
| Super Power|          64|        45|            58.72|          41.28|
|Martial Arts|          23|         6|            79.31|          20.69|
|      Sports|          29|         7|            80.56|          19.44|
|      Shoujo|          74|        11|            8

In [7]:
# ITEM 3: Ver si existe relación acerca de las preferencias de la distribución con respecto al origen (source) de los animes que consumen.
# Archivo necesiario: final_animedataset.csv
# Campos necesarios:
# 	- source
# 	- gender

# Calcular la cantidad de hombres y mujeres por origen (source) de los animes
count_female_male_by_source = final_animedataset.groupBy("source") \
           .agg(
               count(when(col("gender") == "Female", 1)).alias("female_count"),  # Contar las females
               count(when(col("gender") == "Male", 1)).alias("male_count")      # Contar los males
           )

# Agregar las columnas de porcentaje para hombres y mujeres con redondeo a 2 decimales
count_female_male_by_source_with_percen = count_female_male_by_source.withColumn(\
	"female_percentage", round((col("female_count") / (col("female_count") + col("male_count")) * 100), 2)
).withColumn(
    "male_percentage", round((col("male_count") / (col("female_count") + col("male_count")) * 100), 2)
)

# Mostrar los resultados
count_female_male_by_source_with_percen.show(20)


+------------+------------+----------+-----------------+---------------+
|      source|female_count|male_count|female_percentage|male_percentage|
+------------+------------+----------+-----------------+---------------+
|          TV|           1|         1|             50.0|           50.0|
|Visual novel|          36|        11|             76.6|           23.4|
|    Original|         136|        81|            62.67|          37.33|
|       Novel|          43|        15|            74.14|          25.86|
|     Unknown|          23|         0|            100.0|            0.0|
|       Other|           7|         2|            77.78|          22.22|
|       Manga|         374|       106|            77.92|          22.08|
|4-koma manga|          20|         1|            95.24|           4.76|
|       Music|           1|         1|             50.0|           50.0|
|        Game|           7|         3|             70.0|           30.0|
|   Web manga|          11|         1|            9

In [8]:
# ITEM 4: Ver si es posible detectar un patrón referente a qué estudio es más propenso a sacar los tipos de animes que están en estas características.
# Archivo necesiario: 
# Campos necesarios:
# - 



In [9]:
spark.stop()